In [ ]:
import numpy as np
import tensorflow as tf
import neurokit2 as nk
import scipy.signal as signal


class GalaxyEmotionPredictor:
    def __init__(self, wesad_model_paths, deap_model_paths):
        """
        wesad_model_paths: .h5 파일 경로 (Keras load_model 사용)
        deap_model_paths: .tflite 파일 경로 (TFLite Interpreter 사용)
        """
        print("📥 모델 로딩 중...")
        
        # 1. WESAD 모델 로드 (H5 -> Keras)
        self.w_aro = tf.keras.models.load_model(wesad_model_paths['arousal'])
        self.w_val = tf.keras.models.load_model(wesad_model_paths['valence'])
        
        # 2. DEAP 모델 로드 (TFLite -> Interpreter)
        # TFLite는 load_model이 아니라 Interpreter를 써야 합니다.
        self.d_aro_interpreter = tf.lite.Interpreter(model_path=deap_model_paths['arousal'])
        self.d_aro_interpreter.allocate_tensors()
        
        self.d_val_interpreter = tf.lite.Interpreter(model_path=deap_model_paths['valence'])
        self.d_val_interpreter.allocate_tensors()
        
        # TFLite 입출력 정보 미리 가져오기
        self.d_aro_input = self.d_aro_interpreter.get_input_details()
        self.d_aro_output = self.d_aro_interpreter.get_output_details()
        self.d_val_input = self.d_val_interpreter.get_input_details()
        self.d_val_output = self.d_val_interpreter.get_output_details()
        
        print("✅ 모델 로딩 완료! (WESAD: H5, DEAP: TFLite)")

    def _resample(self, data, current_fs, target_fs):
        num_samples = int(len(data) * target_fs / current_fs)
        return signal.resample(data, num_samples)

    # --- TFLite 전용 실행 함수 ---
    def _run_tflite(self, interpreter, input_details, output_details, input_data):
        # TFLite는 float32 타입을 엄격하게 요구합니다.
        input_data = input_data.astype(np.float32)
        
        # 입력 데이터 설정
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # 실행
        interpreter.invoke()
        
        # 결과 가져오기
        output_data = interpreter.get_tensor(output_details[0]['index'])
        return output_data

    # --- WESAD 전처리 (기존 동일) ---
    def preprocess_wesad(self, raw_ppg, fs_watch):
        target_fs = 64
        sig_64 = self._resample(raw_ppg, fs_watch, target_fs)
        ppg_clean = nk.ppg_clean(sig_64, sampling_rate=target_fs)
        
        try:
            features = nk.ppg_analyze(ppg_clean, sampling_rate=target_fs, method='interval')
            feat_values = features.select_dtypes(include=[np.number]).values[0]
            target_feat_size = self.w_aro.input_shape[1][1]
            if len(feat_values) > target_feat_size:
                feat_values = feat_values[:target_feat_size]
            else:
                feat_values = np.pad(feat_values, (0, target_feat_size - len(feat_values)))
        except:
            target_feat_size = self.w_aro.input_shape[1][1]
            feat_values = np.zeros(target_feat_size)

        target_len = target_fs * 60
        if len(ppg_clean) > target_len:
            seg_input = ppg_clean[:target_len]
        else:
            seg_input = np.pad(ppg_clean, (0, target_len - len(ppg_clean)))
            
        seg_input = seg_input.reshape(1, target_len, 1)
        feat_input = feat_values.reshape(1, -1)
        
        return [seg_input, feat_input]

    # --- DEAP 전처리 (TFLite용) ---
    # ==========================================================
    # 2. DEAP 전처리 파이프라인 (수정버전: 차원 자동 맞춤)
    # ==========================================================
    def preprocess_deap(self, raw_ppg, fs_watch):
        target_fs = 128
        sig_128 = self._resample(raw_ppg, fs_watch, target_fs)
        
        # 전처리 (Clean)
        ppg_clean = nk.ppg_clean(sig_128, sampling_rate=target_fs)
        
        # -----------------------------------------------------------
        # 🔥 [핵심 수정] TFLite 모델이 원하는 Shape을 직접 확인해서 맞춥니다.
        # -----------------------------------------------------------
        # self.d_aro_input[0]['shape'] 값은 [1, 8064] (2차원) 일 것입니다.
        required_shape = self.d_aro_input[0]['shape'] 
        target_len = required_shape[1] # 시간 축 길이 (예: 8064)
        
        # 길이 맞추기 (자르거나 0으로 채우기)
        if len(ppg_clean) > target_len:
            model_input = ppg_clean[:target_len]
        else:
            model_input = np.pad(ppg_clean, (0, target_len - len(ppg_clean)))
            
        # ★ 차원 변환: 모델이 원하는 모양 그대로 Reshape 합니다.
        # 기존 코드: model_input.reshape(1, target_len, 1) -> 3차원이라 에러남
        # 수정 코드: model_input.reshape(required_shape)   -> [1, 8064] 2차원으로 변환됨!
        model_input = model_input.reshape(required_shape)
        
        return model_input
    # --- 통합 예측 ---
    def predict_emotion(self, galaxy_ppg_data, watch_fs=25):
        # 1. 전처리
        input_wesad = self.preprocess_wesad(galaxy_ppg_data, watch_fs)
        input_deap = self.preprocess_deap(galaxy_ppg_data, watch_fs)
        
        # 2. Arousal 예측 (WESAD: predict / DEAP: _run_tflite)
        w_prob_a = self.w_aro.predict(input_wesad, verbose=0)
        d_prob_a = self._run_tflite(self.d_aro_interpreter, self.d_aro_input, self.d_aro_output, input_deap)
        
        # 3. Valence 예측
        w_prob_v = self.w_val.predict(input_wesad, verbose=0)
        d_prob_v = self._run_tflite(self.d_val_interpreter, self.d_val_input, self.d_val_output, input_deap)
        
        # 4. 앙상블 평균
        final_prob_a = (w_prob_a + d_prob_a) / 2.0
        final_prob_v = (w_prob_v + d_prob_v) / 2.0
        
        # 5. 결과 판정
        arousal = np.argmax(final_prob_a)
        valence = np.argmax(final_prob_v)
        
        if arousal == 1 and valence == 0: emotion = "😡 Stress"
        elif arousal == 1 and valence == 1: emotion = "😄 Fun"
        elif arousal == 0 and valence == 1: emotion = "😌 Relax"
        elif arousal == 0 and valence == 0: emotion = "🥱 Bored"
        else: emotion = "Unknown"
            
        return emotion, (arousal, valence)
    
if __name__ == "__main__":
    
    wesad_paths = {
        'arousal': r'C:\Users\jwj\Desktop\PPGPROJECT\wesad\wesad_arousal_best_model.h5',
        'valence': r'C:\Users\jwj\Desktop\PPGPROJECT\wesad\wesad_valence_best_model.h5'
    }
    

    deap_paths = {
        'arousal': r'C:\Users\jwj\Desktop\PPGPROJECT\deap\model_ARO.tflite', 
        'valence': r'C:\Users\jwj\Desktop\PPGPROJECT\deap\model_VAL.tflite'
    }
    
    # 1. 예측기 생성
    predictor = GalaxyEmotionPredictor(wesad_paths, deap_paths)
    
    # 2. 갤럭시 워치 데이터 시뮬레이션 (예: 60초 데이터, 25Hz)
    # 실제로는 블루투스로 받은 데이터 리스트가 들어옵니다.
    dummy_watch_data = np.random.rand(25 * 60) 
    
    # 3. 감정 판독
    result_text, scores = predictor.predict_emotion(dummy_watch_data, watch_fs=25)
    print(f"⌚ 갤럭시 워치 측정 결과: {result_text}")

📥 모델 로딩 중...


✅ 모델 로딩 완료! (WESAD: H5, DEAP: TFLite)
⌚ 갤럭시 워치 측정 결과: 😄 Fun
